In [5]:
from data_generator import *
from init_sdr import *
from main import *
from common import *
from measure import *

from cv_model import *
from test_model import *

import json

In [6]:
import ray
import time
import warnings

import torch
import argparse
import ray
from ray import tune
from ray.tune import Trainable
from itertools import product
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import datasets, transforms

import ray
import copy


import time

from sklearn.model_selection import KFold

import argparse

import shutil

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search import ConcurrencyLimiter
from typing import List, Dict

from ray.tune import CLIReporter
import warnings

from ray.tune.search.bayesopt import BayesOptSearch


import math

In [7]:
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import HyperBandScheduler
import shutil

## 일단은 Xaiver로

In [15]:
import time
start_time = time.time()
warnings.filterwarnings("ignore", category=UserWarning)


ray.init(ignore_reinit_error= True)

num_var = 20

hyper_space = {"h_dim1": 1,
             'h_dim2': num_var ,
             "sample_size": 500,
             "num_features": num_var,
             "lambda_0":  tune.grid_search([1e-3,5e-3,1e-2,5e-2]),
             'lr':  tune.grid_search([1e-4, 2.5e-4, 5e-4, 7.5e-4 ,1e-3, 2.5e-3, 5e-3, 7.5e-3, 1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1, 2.5e-1, 5e-1, 7.5e-1]),
             'epoch':  1000,
             'seed': -1,
             "model_num": 'model1',
             "num_cv": 5,
             "or_pen": tune.grid_search(['pro', 'orthogonal']),
             "dim" : 'known'
}

local_dir = 'cv_0419/'
exp_name = "bn_transform"

study = tune.run(
        cv_train_model,
        config=hyper_space, 
        resources_per_trial={"cpu": 2}, 
        metric = 'cv_custom_loss' ,
        mode = 'min',
        raise_on_failed_trial=False, 
        verbose =  False 
)

best_trial = study.get_best_trial(metric="cv_custom_loss", mode = 'min')

ray.shutdown()

end_time = time.time()
execution_time = (end_time - start_time) / 60

print("Execution time:", execution_time, "minutes")
print('-' * 200)

(cv_train_model pid=689773) lambda0: 0.001 lr:  0.005 h_dim2: 20 Epochs 1000 est_dim: 1.0 or_pen pro
(cv_train_model pid=689773) cv_val_mse: 0.9911726713180542 
(cv_train_model pid=689773)  cv_custom_loss 0.991893881559372
(cv_train_model pid=689798) lambda0: 0.005 lr:  0.1 h_dim2: 20 Epochs 1000 est_dim: 1.0 or_pen pro [repeated 53x across cluster]
(cv_train_model pid=689798) cv_val_mse: 1.5876258850097655  [repeated 53x across cluster]
(cv_train_model pid=689798)  cv_custom_loss 1.5880348086357117 [repeated 53x across cluster]
(cv_train_model pid=689773) lambda0: 0.001 lr:  0.5 h_dim2: 20 Epochs 1000 est_dim: 1.0 or_pen pro [repeated 3x across cluster]
(cv_train_model pid=689773) cv_val_mse: 33.316124725341794  [repeated 3x across cluster]
(cv_train_model pid=689773)  cv_custom_loss 33.31615676879883 [repeated 3x across cluster]
(cv_train_model pid=689786) lambda0: 0.05 lr:  0.00025 h_dim2: 20 Epochs 1000 est_dim: 1.0 or_pen orthogonal
(cv_train_model pid=689751) lambda0: 0.005 lr:  

In [18]:
best_config = best_trial.config
print("Best Hyperparameters:", best_config)

Best Hyperparameters: {'h_dim1': 1, 'h_dim2': 20, 'sample_size': 500, 'num_features': 20, 'lambda_0': 0.005, 'lr': 0.025, 'epoch': 1000, 'seed': -1, 'model_num': 'model1', 'num_cv': 5, 'or_pen': 'pro', 'dim': 'known'}


In [19]:
best_loss = best_trial.last_result["cv_custom_loss"]
print("Best Loss:", best_loss)

Best Loss: 0.6303766280412674


In [37]:
trials_df = study.dataframe()

print(trials_df.info())

trials_df.to_csv('fixed_d_cv_results/model8.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cv_val_mse                128 non-null    float64
 1   cv_custom_loss            128 non-null    float64
 2   time_this_iter_s          128 non-null    float64
 3   done                      128 non-null    bool   
 4   timesteps_total           0 non-null      float64
 5   episodes_total            0 non-null      float64
 6   training_iteration        128 non-null    int64  
 7   trial_id                  128 non-null    object 
 8   experiment_id             128 non-null    object 
 9   date                      128 non-null    object 
 10  timestamp                 128 non-null    int64  
 11  time_total_s              128 non-null    float64
 12  pid                       128 non-null    int64  
 13  hostname                  128 non-null    object 
 14  node_ip   

In [12]:
trials_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cv_val_mse                128 non-null    float64
 1   cv_custom_loss            128 non-null    float64
 2   time_this_iter_s          128 non-null    float64
 3   done                      128 non-null    bool   
 4   timesteps_total           0 non-null      float64
 5   episodes_total            0 non-null      float64
 6   training_iteration        128 non-null    int64  
 7   trial_id                  128 non-null    object 
 8   experiment_id             128 non-null    object 
 9   date                      128 non-null    object 
 10  timestamp                 128 non-null    int64  
 11  time_total_s              128 non-null    float64
 12  pid                       128 non-null    int64  
 13  hostname                  128 non-null    object 
 14  node_ip   

In [18]:
import time
start_time = time.time()


ray.init(ignore_reinit_error=True)

warnings.filterwarnings("ignore", category=UserWarning)

num_var = 20

hyper_space = {"h_dim1":1,
             'h_dim2': num_var ,
             "sample_size": 1000,
             "num_features": num_var,
             "lambda_0":  tune.grid_search([1e-3, 5e-3,0.01, 0.05, 0.07, 0.1, 0.5]),
             'lambda_1': tune.grid_search([0.1]),
             'lr':  tune.grid_search([1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 5e-4, 1e-5, 5e-5]),
             'epoch':  1000,
             'seed': -1,
             "model_num": 'model4',
             "num_cv": 5,
             "or_pen": tune.grid_search(['pro', 'orthogonal']),
             "dim" : 'known'
}

local_dir = 'cv_0419/'
exp_name = "bn_transform"

study1 = tune.run(
        cv_train_model,
        config=hyper_space, 
        resources_per_trial={"cpu": 32}, 
        metric = 'cv_custom_loss' ,
        mode = 'min',
        raise_on_failed_trial=False, 
        verbose =  False 
)

best_config2 = study1.get_best_config(metric="cv_custom_loss", mode = 'min')

ray.shutdown()

end_time = time.time()
execution_time = (end_time - start_time) / 60

print("Execution time:", execution_time, "minutes")
print('-' * 200)
print(best_config2)

(cv_train_model pid=1248861) lambda0: 0.001 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248861) cv_val_mse: 1.849621868133545 
(cv_train_model pid=1248861)  cv_custom_loss 1.8534056186676025
(cv_train_model pid=1248865) lambda0: 0.005 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248865) cv_val_mse: 1.8506564378738404 
(cv_train_model pid=1248865)  cv_custom_loss 1.8701382637023927
(cv_train_model pid=1248866) lambda0: 0.01 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248866) cv_val_mse: 1.858734405040741 
(cv_train_model pid=1248866)  cv_custom_loss 1.8924262046813964
(cv_train_model pid=1248866) lambda0: 0.1 lambda1:  0.1 lr:  0.01 h_dim2: 20 Epochs 1000 est_dim:  1.0 or_pen pro
(cv_train_model pid=1248866) cv_val_mse: 1.8294134140014648 
(cv_train_model pid=1248866)  cv_custom_loss 2.1936015605926515
(cv_train_model pid=1248865) lambda0: 0.0